## Setup Connection to Drive

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

In [2]:
import os
rootPath = "/content/drive/My Drive/BuildingDamage_Model"
os.chdir(rootPath)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [4]:
!grep MemTotal /proc/meminfo

MemTotal:       26751688 kB


## Import Packages

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [ ]:
!pip install geojsonio

In [10]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import rtree
import requests
import geojsonio

In [ ]:
! pip install planet

In [ ]:
! pip install rasterio

In [124]:
import os
from pprint import pprint

import fiona
import matplotlib.pyplot as plt
from planet import api
from planet.api.utils import write_to_file
import rasterio
from rasterio import features as rfeatures
from rasterio.enums import Resampling
from rasterio.plot import show
import shapely
from shapely.geometry import shape as sshape

## Access Planet API 
### Authentication

### 

In [ ]:
!git clone https://github.com/planetlabs/notebooks.git
!cd notebooks

In [ ]:
!cd notebooks
!docker pull planetlabs/notebooks
!docker tag planetlabs/notebooks planet-notebooks

In [125]:
# Import the os module in order to access environment variables
import os

# If you're following along with this notebook, you can enter your API Key on the following line, and uncomment it:
os.environ['PL_API_KEY']='bd2b0d6fe4b44447a799648ecd0fba3f'

# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('bd2b0d6fe4b44447a799648ecd0fba3f')

API_KEY = os.environ.get('PL_API_KEY', 'bd2b0d6fe4b44447a799648ecd0fba3f')

In [92]:
import json
import requests

In [93]:
# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [94]:
# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "bd2b0d6fe4b44447a799648ecd0fba3f")

In [95]:
# Make a GET request to the Planet Data API
res = session.get(URL)

In [96]:
# Response status code
res.status_code

200

In [97]:
# Response Body
res.text

'{"_links": {"_self": "https://api.planet.com/data/v1/", "asset-types": "https://api.planet.com/data/v1/asset-types/", "item-types": "https://api.planet.com/data/v1/item-types/", "spec": "https://api.planet.com/data/v1/spec"}}'

In [98]:
# Print formatted JSON response
p(res.json())

{
  "_links": {
    "_self": "https://api.planet.com/data/v1/",
    "asset-types": "https://api.planet.com/data/v1/asset-types/",
    "item-types": "https://api.planet.com/data/v1/item-types/",
    "spec": "https://api.planet.com/data/v1/spec"
  }
}


In [99]:
# Print the value of the item-types key from _links
print(res.json()["_links"]["item-types"])

https://api.planet.com/data/v1/item-types/


In [100]:
# Setup the stats URL
stats_url = "{}/stats".format(URL)

# Print the stats URL
print(stats_url)

https://api.planet.com/data/v1/stats


In [101]:
#Specify the sensors/satellites or "item types" to include in our results
item_types = ["PSScene3Band", "REOrthoTile"]

## Filters

#### Date

In [105]:
# Create filter object for all imagery captured between 2013-01-01 and present.
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2012-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
    }
}

#### cloud

In [107]:
cloud_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lt": 0.2,
    "gt": 0.1
  }
}

#### permission

In [108]:
permission_filter = {
  "type": "PermissionFilter",
  "config": ["assets.analytic:download"]
}

####geometry

In [111]:
# Search for imagery that only intersects with 40N, 90W

# Setup GeoJSON 
geom = {
    "type": "Point",
    "coordinates": [
        -90,
         40
    ]
}

# Setup Geometry Filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": "POINT(-90 40)"
}

instrument

In [112]:
# Search for imagery only from PlanetScope satellites that have a PS2 telescope

# Setup item types
item_types = ["PSScene3Band"]

# Setup a filter for instrument type
instrument_filter = {
    "type": "StringInFilter",
    "field_name": "instrument",
    "config": ["PS2"]
}


## Combine all filters

In [ ]:
# Setup an "AND" logical filter
and_filter = {
    "type": "AndFilter",
    "config": [cloud_filter, date_filter, instrument_filter, permission_filter]
}

# Print the logical filter
p(and_filter)

## Request

In [ ]:
request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : and_filter
}

# Send the POST request to the API stats endpoint
res=session.post(stats_url, json=request)

# Print response
p(res.json())

ACCESS Building Feed ID

In [126]:
analytics_client = api.ClientV1(api_key=API_KEY)

In [127]:
# uncomment to get feed ids
feeds = analytics_client.list_analytic_feeds({}).get()
for d in feeds['data']:
    print('{} ({}):\n\r{}\n\r'.format(d['id'], d['created'], d['description']))

In [137]:
# # uncomment to get subscription ids
FEED_ID = 'b442c53b-fc72-4bee-bab4-0b7aa318ccd9'
subscriptions = analytics_client.list_analytic_subscriptions(FEED_ID).get()
for d in subscriptions['data']:
    print('{} ({}):\n\r{}\n\r'.format(d['id'], d['created'], d['title']))

In [130]:
subscriptions

{'data': [],
 'links': [{'href': 'https://api.planet.com/analytics/subscriptions?feedID=b442c53b-fc72-4bee-bab4-0b7aa318ccd9',
   'rel': 'self',
   'type': 'application/json'}]}

In [135]:
# building footprints in Sazgin, Turkey
SUBSCRIPTION_ID = '02c4f912-090f-45aa-a18b-ac4a55e4b9ba'

In [ ]:
#Get subscription details
subscription_info = analytics_client.get_subscription_info(SUBSCRIPTION_ID).get()
pprint(subscription_info)